In [ ]:
# Import libraries
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import timm

# Define file paths
LABELS_CSV = './data/train_labels.csv'
TRAIN_PATH = './data/train'

# Label encoding dictionaries
label2id = {"Alluvial soil": 0, "Black Soil": 1, "Clay soil": 2, "Red soil": 3}
id2label = {v: k for k, v in label2id.items()}


In [ ]:
class SoilDataset(Dataset):
    def __init__(self, df, image_dir, transform=None, is_test=False):
        self.df = df
        self.image_dir = image_dir
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row['image_id'])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        if self.is_test:
            return image, row['image_id']
        else:
            label = label2id[row['soil_type']]
            return image, label


In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(0.3, 0.3, 0.3, 0.05),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])


In [ ]:
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, smoothing=0.1):
        super().__init__()
        self.smoothing = smoothing
        self.confidence = 1.0 - smoothing

    def forward(self, x, target):
        log_probs = torch.nn.functional.log_softmax(x, dim=-1)
        true_dist = torch.zeros_like(log_probs)
        true_dist.fill_(self.smoothing / (x.size(1) - 1))
        true_dist.scatter_(1, target.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * log_probs, dim=-1))


In [ ]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0

    for images, labels in tqdm(loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)


In [ ]:
def validate(model, loader, device):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Validating", leave=False):
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.numpy())

    report = classification_report(all_labels, all_preds, output_dict=True, zero_division=0)
    per_class_f1 = [report[str(i)]['f1-score'] for i in range(4)]
    return accuracy_score(all_labels, all_preds), per_class_f1, min(per_class_f1)


In [ ]:
def run_kfold_training(df, k=3, epochs=10):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    all_models = []
    all_min_f1 = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['soil_type'])):
        print(f"\n----- Fold {fold+1}/{k} -----")
        train_df = df.iloc[train_idx].reset_index(drop=True)
        val_df = df.iloc[val_idx].reset_index(drop=True)

        train_dataset = SoilDataset(train_df, TRAIN_PATH, transform=train_transform)
        val_dataset = SoilDataset(val_df, TRAIN_PATH, transform=val_transform)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32)

        model = timm.create_model('convnext_base', pretrained=True, num_classes=4)
        model.to(device)

        criterion = LabelSmoothingCrossEntropy(0.1)
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

        best_min_f1 = 0
        for epoch in range(epochs):
            print(f"Epoch {epoch+1}/{epochs}")
            train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
            val_acc, f1s, min_f1 = validate(model, val_loader, device)
            print(f"Train Loss: {train_loss:.4f}, Val Acc: {val_acc:.4f}, Min F1: {min_f1:.4f}")

            if min_f1 > best_min_f1:
                best_min_f1 = min_f1
                torch.save(model.state_dict(), f"best_model_3fold_fold{fold+1}.pth")

            scheduler.step()

        all_min_f1.append(best_min_f1)
        all_models.append(f"best_model_3fold_fold{fold+1}.pth")

    print(f"\nAverage Min F1 across {k} folds: {np.mean(all_min_f1):.4f}")
    return all_models

if __name__ == '__main__':
    labels_df = pd.read_csv(LABELS_CSV)
    print("Starting 3-fold training...")
    model_paths = run_kfold_training(labels_df, k=3, epochs=10)
